## Scraping Detik

In [62]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from datetime import datetime
import threading
import re

In [63]:
# jumlah_index = 1
threads_link = []
links = []
results = []
keywords = 'jokowi'
since_time='01/10/2023'
until_time='11/10/2023'



In [64]:
def cek_totalPagination(keywords, since_time, until_time):
    url = f"https://www.detik.com/search/searchall?query={keywords}&siteid=2&sortby=time&fromdatex={since_time}&todatex={until_time}&page=1"
    
    response = requests.get(url) 
    soup = BeautifulSoup(response.text, "html.parser")
    cek = soup.find('span', {"class": "fl text"})
    hasil_text = cek.get_text()

    # Use regular expression to extract the number
    result = re.search(r'\d+', hasil_text)
    if result:
        hasil_number = int(result.group(0))
        hasil_divided = hasil_number / 9  # Divide the number by 9
        rounded_result = round(hasil_divided)  # Round to the nearest whole number
        return rounded_result
    else:
        return 0  # Return 0 if no number was found


In [65]:
jumlah_index = cek_totalPagination(keywords, since_time, until_time)
print(f"Total results: {jumlah_index}")

Total results: 97


In [66]:
def scrape_links(page_number,keywords, since_time, until_time):
    url = f"https://www.detik.com/search/searchall?query={keywords}&siteid=2&sortby=time&fromdatex={since_time}&todatex={until_time}&page={page_number}"
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")
    articles = soup.find_all('article')
    
    page_links = []
    for article in articles:
        link = article.find('a')['href']
        page_links.append(link)
    
    print(f"Scraped {len(page_links)} links from page {page_number}")
    return page_links

In [67]:
for page_number in range(1, jumlah_index + 1):
    thread = threading.Thread(target=lambda p=page_number: links.extend(scrape_links(p,keywords,since_time,until_time)))
    thread.start()
    threads_link.append(thread)

for thread in threads_link:
    thread.join()
print("Total Links:", len(links))


Scraped 9 links from page 6
Scraped 9 links from page 12
Scraped 9 links from page 7
Scraped 9 links from page 8
Scraped 9 links from page 3
Scraped 9 links from page 31
Scraped 9 links from page 10
Scraped 9 links from page 23
Scraped 9 links from page 17
Scraped 9 links from page 24Scraped 9 links from page 22

Scraped 9 links from page 27
Scraped 9 links from page 15
Scraped 9 links from page 11
Scraped 9 links from page 16
Scraped 9 links from page 30
Scraped 9 links from page 29
Scraped 9 links from page 4
Scraped 9 links from page 18
Scraped 9 links from page 38
Scraped 9 links from page 26
Scraped 9 links from page 2
Scraped 9 links from page 19
Scraped 9 links from page 32
Scraped 9 links from page 9
Scraped 9 links from page 42
Scraped 9 links from page 33
Scraped 9 links from page 25
Scraped 9 links from page 35
Scraped 9 links from page 5
Scraped 9 links from page 1
Scraped 9 links from page 45
Scraped 9 links from page 13
Scraped 9 links from page 61
Scraped 9 links from pa

In [68]:
def scrape_url(url,keywords):
    try:
        response = requests.get(url)
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')
            
            
            # Judul Berita
            title_elem = soup.find('h1', {"class": "detail__title"})
            if title_elem:
                title_text = title_elem.text.strip()
            else:
                title_text = "Title not found"
            # Author berita
            author_elem = soup.find('div', {"class": "detail__author"})
            if author_elem:
                author_text = author_elem.get_text()
                author_text = author_text.split('-')[0].strip()
            else:
                author_text = "Author not found"     
            # tanggal berita
            date_elem = soup.find('div', {"class": "detail__date"})
            if date_elem:
                date_text = date_elem.text.strip()
            else:
                date_text = "Date not found"
            #     # Category berita
            category_elements = soup.find('div',{"class": "page__breadcrumb"})
            if category_elements:
                category_text = category_elements.find('a',{"dtr-sec": "breadcrumbkanal"})
                category_text= category_text.text.strip()
            else:
                category_text = "Category not found"
            #     # Content Berita
            body_elem = soup.find('div', {"class": "detail__body"})
            
            if body_elem:
                content_elem = body_elem.find_all('p')
                content_text = ""
                for p in content_elem:
                    content_text += p.text.strip() + "\n"
                
                if content_text.strip():
                    content_text=content_text
                else:
                    content_text ="Content not found"
            else:
              content_text ="Content not found"

            nama_berita_match = re.search(r'https://(?:www\.)?([a-zA-Z0-9.-]+)\.com', url)
            if nama_berita_match:
                nama_berita = nama_berita_match.group(1)
            else:
                nama_berita = "Nama_berita not found"
            results.append({'title': title_text,
                            'keywords':keywords,
                            'author' : author_text,
                            'category':category_text,
                            'date': date_text,
                            'content' : content_text,
                            'nama_berita' : nama_berita,
                            'link' : url})
        else:
            print(f"Failed to retrieve data from {url}")

    except requests.exceptions.RequestException as e:
        print(f"Error fetching URL '{url}': {e}")
    except Exception as e:
        print(f"Error processing URL '{url}': {e}")

In [69]:
threads = []
for url in links:
    thread = threading.Thread(target=scrape_url, args=(url,keywords))
    thread.start()
    threads.append(thread)
    
for thread in threads:
    thread.join()

In [70]:
df = pd.DataFrame(results)
print('hasil scrapping',len(results))
df.head(10)

hasil scrapping 873


,title,keywords,author,category,date,content,nama_berita,link
0,"Adu Perspektif: Menanti Cawapres Pilihan Mega,...",jokowi,Syailendra Hafiz Wiratama,Berita,"Rabu, 11 Okt 2023 13:38 WIB",Pendaftaran capres-cawapres untuk Pilpres 2024...,news.detik,https://news.detik.com/berita/d-6976511/adu-pe...
1,Tentang LRT dan Teknologi yang Dipakai,jokowi,Baladan Hadza Firosya,Detikpedia,"Rabu, 11 Okt 2023 13:30 WIB",Light Rail Transit (LRT) secara resmi diresmik...,detik,https://www.detik.com/edu/detikpedia/d-6976368...
2,Luhut Sampaikan Terima Kasih Jokowi Kirim Tim...,jokowi,Ignacio Geordi Oswaldo,Berita Ekonomi Bisnis,"Selasa, 10 Okt 2023 19:50 WIB",Menteri Koordinator Kemaritiman dan Investasi ...,finance.detik,https://finance.detik.com/berita-ekonomi-bisni...
3,Gerindra soal Cawapres Prabowo: Partai Koalisi...,jokowi,Author not found,Category not found,"19,586 Views | Selasa, 10 Okt 2023 19:35 WIB","Anggota Dewan Pembina Gerindra, Andre Rosiade ...",news.detik,https://news.detik.com/detiktv/d-6975521/gerin...
4,"Relawan Samawi Lanjut Ketemu Prabowo, Pakai Ja...",jokowi,Author not found,Category not found,"6,041 Views | Sabtu, 07 Okt 2023 20:16 WIB",Relawan Solidaritas Ulama Muda Jokowi (Samawi)...,news.detik,https://news.detik.com/detiktv/d-6970531/relaw...
5,Massa Demo Bela Palestina di Kedubes AS Bubark...,jokowi,Annisa Aulia Rahim,Berita,"Rabu, 11 Okt 2023 12:30 WIB","Massa Aqsa Working Group (AWG), yang menggelar...",news.detik,https://news.detik.com/berita/d-6976376/massa-...
6,Jokowi dan (Warisan) Kereta Cepat Jakarta-Bandung,jokowi,Ari Putra Utama,Kolom,"Rabu, 11 Okt 2023 13:08 WIB",[Gambas:Video 20detik]\nADVERTISEMENT\nSCROLL ...,news.detik,https://news.detik.com/kolom/d-6976456/jokowi-...
7,Kebijakan Pakai Lahan hingga 190 Tahun Bisa Ga...,jokowi,Shafira Cendra Arini,Infrastruktur,"Selasa, 10 Okt 2023 16:10 WIB",Revisi Undang-Undang (UU) Nomor 3 tahun 2022 t...,finance.detik,https://finance.detik.com/infrastruktur/d-6975...
8,Bagaimana Kondisi Rempang Saat Ini? Begini Pen...,jokowi,Danang Sugianto,Infrastruktur,"Minggu, 08 Okt 2023 11:07 WIB",Pulau Rempang sempat menjadi sorotan khalayak ...,finance.detik,https://finance.detik.com/infrastruktur/d-6971...
9,Relawan Samawi Merapat ke Rumah Prabowo Usai R...,jokowi,Mulia Budi,Pemilu,"Sabtu, 07 Okt 2023 19:52 WIB",Solidaritas Ulama Muda Jokowi (Samawi) merapat...,news.detik,https://news.detik.com/pemilu/d-6970492/relawa...


In [71]:
filtered_df = df[df['content'].str.contains(keywords, case=False)|
                 df['title'].str.contains(keywords, case=False)]
print('hasil filter',len(filtered_df))
filtered_df.head(10)

hasil filter 839


,title,keywords,author,category,date,content,nama_berita,link
0,"Adu Perspektif: Menanti Cawapres Pilihan Mega,...",jokowi,Syailendra Hafiz Wiratama,Berita,"Rabu, 11 Okt 2023 13:38 WIB",Pendaftaran capres-cawapres untuk Pilpres 2024...,news.detik,https://news.detik.com/berita/d-6976511/adu-pe...
2,Luhut Sampaikan Terima Kasih Jokowi Kirim Tim...,jokowi,Ignacio Geordi Oswaldo,Berita Ekonomi Bisnis,"Selasa, 10 Okt 2023 19:50 WIB",Menteri Koordinator Kemaritiman dan Investasi ...,finance.detik,https://finance.detik.com/berita-ekonomi-bisni...
3,Gerindra soal Cawapres Prabowo: Partai Koalisi...,jokowi,Author not found,Category not found,"19,586 Views | Selasa, 10 Okt 2023 19:35 WIB","Anggota Dewan Pembina Gerindra, Andre Rosiade ...",news.detik,https://news.detik.com/detiktv/d-6975521/gerin...
4,"Relawan Samawi Lanjut Ketemu Prabowo, Pakai Ja...",jokowi,Author not found,Category not found,"6,041 Views | Sabtu, 07 Okt 2023 20:16 WIB",Relawan Solidaritas Ulama Muda Jokowi (Samawi)...,news.detik,https://news.detik.com/detiktv/d-6970531/relaw...
5,Massa Demo Bela Palestina di Kedubes AS Bubark...,jokowi,Annisa Aulia Rahim,Berita,"Rabu, 11 Okt 2023 12:30 WIB","Massa Aqsa Working Group (AWG), yang menggelar...",news.detik,https://news.detik.com/berita/d-6976376/massa-...
6,Jokowi dan (Warisan) Kereta Cepat Jakarta-Bandung,jokowi,Ari Putra Utama,Kolom,"Rabu, 11 Okt 2023 13:08 WIB",[Gambas:Video 20detik]\nADVERTISEMENT\nSCROLL ...,news.detik,https://news.detik.com/kolom/d-6976456/jokowi-...
7,Kebijakan Pakai Lahan hingga 190 Tahun Bisa Ga...,jokowi,Shafira Cendra Arini,Infrastruktur,"Selasa, 10 Okt 2023 16:10 WIB",Revisi Undang-Undang (UU) Nomor 3 tahun 2022 t...,finance.detik,https://finance.detik.com/infrastruktur/d-6975...
8,Bagaimana Kondisi Rempang Saat Ini? Begini Pen...,jokowi,Danang Sugianto,Infrastruktur,"Minggu, 08 Okt 2023 11:07 WIB",Pulau Rempang sempat menjadi sorotan khalayak ...,finance.detik,https://finance.detik.com/infrastruktur/d-6971...
9,Relawan Samawi Merapat ke Rumah Prabowo Usai R...,jokowi,Mulia Budi,Pemilu,"Sabtu, 07 Okt 2023 19:52 WIB",Solidaritas Ulama Muda Jokowi (Samawi) merapat...,news.detik,https://news.detik.com/pemilu/d-6970492/relawa...
10,Warga Terdampak Gempa Cianjur Dapat Ganti Ruma...,jokowi,Dana Aditiasari,Berita,"Rabu, 11 Okt 2023 07:28 WIB",Masyarakat terdampak bencana gempa bumi di Kab...,detik,https://www.detik.com/properti/berita/d-697587...


Save Data

In [72]:
current_datetime = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
excel_file_name = f'../tempat_simpan_nasional/detik_{keywords}_{current_datetime}_Jangka_waktu_{since_time}_{until_time}.xlsx'
filtered_df.to_excel(excel_file_name, index=False)

print(f'Data has been saved to {excel_file_name}')

Data has been saved to ../tempat_simpan_nasional/detik_jokowi_2023-10-11_15-49-04.xlsx
